In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

In [3]:
year = 2022
day = 19

19

In [4]:
input = get_input(year, day)

"Blueprint 1: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 19 clay. Each geode robot costs 2 ore and 12 obsidian.\nBlueprint 2: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 19 " ⋯ 4293 bytes ⋯ " robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 2 ore and 12 obsidian.\nBlueprint 30: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 8 clay. Each geode robot costs 2 ore and 15 obsidian."

In [5]:
parsed_input = parse_input(input)

30-element Vector{SubString{String}}:
 "Blueprint 1: Each ore robot cos" ⋯ 98 bytes ⋯ "ot costs 2 ore and 12 obsidian."
 "Blueprint 2: Each ore robot cos" ⋯ 97 bytes ⋯ "bot costs 2 ore and 9 obsidian."
 "Blueprint 3: Each ore robot cos" ⋯ 98 bytes ⋯ "ot costs 3 ore and 10 obsidian."
 "Blueprint 4: Each ore robot cos" ⋯ 98 bytes ⋯ "ot costs 4 ore and 15 obsidian."
 "Blueprint 5: Each ore robot cos" ⋯ 98 bytes ⋯ "ot costs 3 ore and 15 obsidian."
 "Blueprint 6: Each ore robot cos" ⋯ 97 bytes ⋯ "ot costs 3 ore and 19 obsidian."
 "Blueprint 7: Each ore robot cos" ⋯ 97 bytes ⋯ "ot costs 4 ore and 17 obsidian."
 "Blueprint 8: Each ore robot cos" ⋯ 98 bytes ⋯ "ot costs 3 ore and 16 obsidian."
 "Blueprint 9: Each ore robot cos" ⋯ 97 bytes ⋯ "bot costs 2 ore and 9 obsidian."
 "Blueprint 10: Each ore robot co" ⋯ 97 bytes ⋯ "bot costs 3 ore and 7 obsidian."
 ⋮
 "Blueprint 22: Each ore robot co" ⋯ 99 bytes ⋯ "ot costs 4 ore and 20 obsidian."
 "Blueprint 23: Each ore robot co" ⋯ 99 bytes ⋯ "ot costs

In [6]:
sample = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

"Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.\nBlueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."

In [7]:
parsed_sample = parse_input(sample)

2-element Vector{SubString{String}}:
 "Blueprint 1: Each ore robot cos" ⋯ 97 bytes ⋯ "bot costs 2 ore and 7 obsidian."
 "Blueprint 2: Each ore robot cos" ⋯ 97 bytes ⋯ "ot costs 3 ore and 12 obsidian."

In [8]:
function preprocess(input)
    map(input) do line
        parts = split(line, "Each ")[2:5]
        map(parts) do part
            items = split(part, " costs ")[2]
            items = rstrip(items)
            @assert items[end] == '.'
            items = items[1:end-1]
            items = split(items, " and ")
            items = map(items) do item
                item = split(item, " ")
                @assert length(item) == 2
                parse(Int, item[1])
            end
            tuple(items...)
        end
    end
end

preprocess (generic function with 1 method)

In [9]:
function solve(bp, T, ORE, CLAY, OBSIDIAN, MAXORE, MAXCLAY, MAXOBSIDIAN)
    # (ra, rb, rc, a, b, c, t)
    dp = OffsetArray(fill(-1, 1:ORE, 0:CLAY, 0:OBSIDIAN, 0:MAXORE, 0:MAXCLAY, 0:MAXOBSIDIAN, 0:T), 1:ORE, 0:CLAY, 0:OBSIDIAN, 0:MAXORE, 0:MAXCLAY, 0:MAXOBSIDIAN, 0:T)
    dp[1, 0, 0, 0, 0, 0, 0] = 0
    for t in 0:T - 1
        for ra in 1:ORE, rb in 0:CLAY, rc in 0:OBSIDIAN, a in 0:MAXORE, b in 0:MAXCLAY, c in 0:MAXOBSIDIAN
            now = CartesianIndex(ra, rb, rc, a, b, c, t)
            if dp[now] == -1
                continue
            end

            if a >= bp[1][1] && ra < ORE
                idx = CartesianIndex(ra+1, rb, rc, min(a - bp[1][1] + ra, MAXORE), min(b + rb, MAXCLAY), min(c + rc, MAXOBSIDIAN), t+1)
                dp[idx] = max(dp[idx], dp[now])
            end

            if a >= bp[2][1] && rb < CLAY
                idx = CartesianIndex(ra, rb + 1, rc, min(a - bp[2][1] + ra, MAXORE), min(b + rb, MAXCLAY), min(c + rc, MAXOBSIDIAN), t+1)
                dp[idx] = max(dp[idx], dp[now])
            end

            if a >= bp[3][1] && b >= bp[3][2] && rc < OBSIDIAN
                idx = CartesianIndex(ra, rb, rc + 1, min(a - bp[3][1] + ra, MAXORE), min(b - bp[3][2] + rb, MAXCLAY), min(c + rc, MAXOBSIDIAN), t+1)
                dp[idx] = max(dp[idx], dp[now])
            end

            if a >= bp[4][1] && c >= bp[4][2]
                idx = CartesianIndex(ra, rb, rc, min(a - bp[4][1] + ra, MAXORE), min(b + rb, MAXCLAY), min(c - bp[4][2] + rc, MAXOBSIDIAN), t+1)
                dp[idx] = max(dp[idx], dp[now] + T - 1 - t)
            end

            idx =  CartesianIndex(ra, rb, rc, min(a + ra, MAXORE), min(b + rb, MAXCLAY), min(c + rc, MAXOBSIDIAN), t+1)
            dp[idx] = max(dp[idx], dp[now])
        end 
    end

    return maximum(dp)
end

solve (generic function with 1 method)

In [10]:
function part_one(input)
    bps = preprocess(input)
    return sum(map(enumerate(bps)) do (i, bp)
        T = 24
        ORE = max(bp[1][1], bp[2][1], bp[3][1], bp[4][1])
        CLAY = bp[3][2]
        OBSIDIAN = bp[4][2]

        MAXORE = 2 * ORE
        MAXCLAY = 2 * CLAY
        MAXOBSIDIAN = 2 * OBSIDIAN
        i * solve(bp, T, ORE, CLAY, OBSIDIAN, MAXORE, MAXCLAY, MAXOBSIDIAN)
    end)
end

part_one (generic function with 1 method)

In [11]:
part_one(parsed_sample)

33

In [12]:
part_one_ans = part_one(parsed_input)

1589

In [13]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 19 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3336 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [14]:
function part_two(input)
    bps = preprocess(input)
    return prod(map(bps) do bp
        T = 32
        ORE = max(bp[1][1], bp[2][1], bp[3][1], bp[4][1])
        CLAY = bp[3][2]
        OBSIDIAN = bp[4][2]

        MAXORE = 2 * ORE
        MAXCLAY = 2 * CLAY
        MAXOBSIDIAN = 2 * OBSIDIAN
        solve(bp, T, ORE, CLAY, OBSIDIAN, MAXORE, MAXCLAY, MAXOBSIDIAN)
    end)
end

part_two (generic function with 1 method)

In [15]:
part_two(parsed_sample)

3472

In [16]:
part_two_ans = part_two(parsed_input[1:3])

29348

In [17]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 19 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3524 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"